# LGBM

In [1]:
import pandas as pd
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns

## 1. 데이터 로딩

In [2]:
data_dir = '../data' # 경로는 상황에 맞춰서 수정해주세요!
csv_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)

dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

df = pd.read_csv(csv_file_path, dtype=dtype, parse_dates=['Timestamp']) 

In [3]:
df['testId'].nunique()

1537

## 2. Feature Engineering

In [4]:
def feature_engineering(df):
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']

    ### 소요시간
    diff = df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    diff = diff.fillna(pd.Timedelta(seconds=0))
    diff = diff['Timestamp'].apply(lambda x: x.total_seconds())
    df['elapsed'] = diff
    elapsed_median = df[df['elapsed'] < 3600]['elapsed'].median()
    def elapsed_preprocessing(x):
        if x >= 3600 or x == 0:
            return elapsed_median
        return x
    df['elapsed'] = df['elapsed'].apply(elapsed_preprocessing)

    ### 문제 푼 시간 추가
    df['Time'] = df['Timestamp'].dt.hour

    ### 연속적으로 푼 태그 정보 추가
    df['continuous_tag'] = (df.groupby(['userID', 'KnowledgeTag']).cumcount() + 1).astype(int)
    
    ### 유저 별 평균 소요 시간 추가
    def percentile(x):
        return np.sum(x) / len(x)

    user_time_groupby = df.groupby('userID').agg({
        'elapsed' : percentile
    }).reset_index()

    user_time_groupby.columns = ['userID', 'userTime']

    df = pd.merge(df, user_time_groupby, on=['userID'], how="left")
    
    
    ### 사용자가 마지막으로 푼 문항과 같은 지식 태그를 가진 문항을 푼 시간과 
    # 사용자가 마지막으로 푼 문항과의 시간 간격을 계산
    df['time_since_last_question'] = df.groupby('userID')['Timestamp'].diff().apply(lambda x: x.total_seconds()) 
    df['time_since_last_question'] = df.groupby(['userID', 'KnowledgeTag'])['time_since_last_question'].shift(1) # elapsed
    # 'similar_question_answered' 컬럼을 생성합니다.
    # 사용자가 마지막으로 푼 문항과 같은 지식 태그를 가진 문항을 푼 지 여부
    df['similar_question_answered'] = df['time_since_last_question'].apply(lambda x: 1 if x is not None and x <= 60*60*24*3 else 0)
    df = df.drop(['time_since_last_question'], axis=1)
    
    ### 사용자가 문항을 푼 시간대 (오전/오후/밤/새벽)
    time = df['Timestamp'].dt.hour
    def timezone_preprocessing(x):
        if 7 <= x < 13: # 7~12
            x = 0
        elif 13 <= x < 20: # 13 ~ 19
            x = 1 
        elif 20 <= x < 24: # 20 ~ 23
            x = 2
        else:
            x = 3
        return x
    df['Timezone'] = time.apply(timezone_preprocessing)
    
    ### 주말 여부
    time = df['Timestamp'].dt.day_name()
    def isWeekend_preprocessing(x):
        if x =='Saturday' or x =='Sunday':
            x = 1
        else:
            x = 0
        return x
    df['isWeekend'] = time.apply(isWeekend_preprocessing)
    
    ### 문항이 속한 태그 중 사용자가 이전에 푼 문항에서 맞춘 비율
    df['user_tag_cum_acc'] = df.groupby(['userID', 'KnowledgeTag'])['answerCode'].transform(lambda x: x.expanding().mean().shift(1))
    
    
    
    
    return df

In [5]:
def feature_engineering_train(df):
    global correct_t, correct_k, question_accuracy
    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
    correct_t.columns = ["test_mean", 'test_sum']
    correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
    correct_k.columns = ["tag_mean", 'tag_sum']

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    ### 사용자가 이전에 푼 문항과 동일한 지식 태그를 포함한 문항일 경우 정답률이 높아지는 경향이 있는 지 여부
    # 문항별 정답률 계산
    question_accuracy = df.groupby('assessmentItemID')['answerCode'].mean()
    
    # 이전에 푼 문항과 동일한 태그를 가진 문항일 경우 1, 그렇지 않을 경우 0으로 구성된 새로운 변수 생성
    df['similar_tag_answered'] = df.groupby('userID')['KnowledgeTag'].apply(lambda x: x.eq(x.shift()).astype(int)).reset_index(drop=True)
    # 이전에 푼 문항과 동일한 태그를 가진 문항일 경우 그 문항의 정답률 계산
    df['similar_tag_accuracy'] = df['assessmentItemID'].map(question_accuracy) * df['similar_tag_answered']
    # 만약 이전에 푼 문항과 동일한 태그를 가진 문항이 없을 경우에는 정답률이 0이 되도록 함
    df['similar_tag_accuracy'].fillna(0, inplace=True)
    df = df.drop(['similar_tag_answered'], axis=1)
    
    return df

In [6]:
def feature_engineering_test(df):
    global correct_t, correct_k, question_accuracy
    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    # 이전에 푼 문항과 동일한 태그를 가진 문항일 경우 1, 그렇지 않을 경우 0으로 구성된 새로운 변수 생성
    df['similar_tag_answered'] = df.groupby('userID')['KnowledgeTag'].apply(lambda x: x.eq(x.shift()).astype(int)).reset_index(drop=True)
    # 이전에 푼 문항과 동일한 태그를 가진 문항일 경우 그 문항의 정답률 계산
    df['similar_tag_accuracy'] = df['assessmentItemID'].map(question_accuracy) * df['similar_tag_answered']
    # 만약 이전에 푼 문항과 동일한 태그를 가진 문항이 없을 경우에는 정답률이 0이 되도록 함
    df['similar_tag_accuracy'].fillna(0, inplace=True)
    df = df.drop(['similar_tag_answered'], axis=1)
    
    return df

In [7]:
df = feature_engineering(df)
df = feature_engineering_train(df)
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,elapsed,...,userTime,similar_question_answered,Timezone,isWeekend,user_tag_cum_acc,test_mean,test_sum,tag_mean,tag_sum,similar_tag_accuracy
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,NaN,0,NaN,28.0,...,49.851007,0,3,0,NaN,0.947683,1268,0.955022,637,0.000000
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,1,1.0,3.0,...,49.851007,0,3,0,NaN,0.947683,1268,0.913187,3040,0.000000
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,2.0,2,1.0,8.0,...,49.851007,1,3,0,1.0,0.947683,1268,0.913187,3040,0.910314
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,3.0,3,1.0,7.0,...,49.851007,1,3,0,1.0,0.947683,1268,0.913187,3040,0.968610
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,4.0,4,1.0,7.0,...,49.851007,1,3,0,1.0,0.947683,1268,0.913187,3040,0.941704


### Feature Engineering 연습장

In [9]:
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,elapsed,...,userTime,similar_question_answered,Timezone,isWeekend,user_tag_cum_acc,test_mean,test_sum,tag_mean,tag_sum,similar_tag_accuracy
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,NaN,0,NaN,28.0,...,49.851007,0,3,0,NaN,0.947683,1268,0.955022,637,0.000000
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,1,1.0,3.0,...,49.851007,0,3,0,NaN,0.947683,1268,0.913187,3040,0.000000
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,2.0,2,1.0,8.0,...,49.851007,1,3,0,1.0,0.947683,1268,0.913187,3040,0.910314
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,3.0,3,1.0,7.0,...,49.851007,1,3,0,1.0,0.947683,1268,0.913187,3040,0.968610
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,4.0,4,1.0,7.0,...,49.851007,1,3,0,1.0,0.947683,1268,0.913187,3040,0.941704


## 3. Train/Test 데이터 셋 분리

In [10]:
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함
random.seed(42)
def custom_train_test_split(df, ratio=0.7, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)


    train = df[df['userID'].isin(user_ids)]
    test = df[df['userID'].isin(user_ids) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    return train, test

In [11]:
# 유저별 분리
train, test = custom_train_test_split(df)

# 사용할 Feature 설정
### 추가할 Feature 뒤에 넣기
FEATS = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum', 'elapsed', 'Time', 'continuous_tag', 'userTime' , 'similar_question_answered', 'isWeekend', 'user_tag_cum_acc', 'similar_tag_accuracy']

# X, y 값 분리
y_train = train['answerCode']
train = train.drop(['answerCode'], axis=1)

y_test = test['answerCode']
test = test.drop(['answerCode'], axis=1)

In [12]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np

## 4. 훈련 및 검증

In [14]:
# Baseline 성능 추출

from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

lgb_train = LGBMClassifier().fit(train[FEATS], y_train)
lgb_test = LGBMClassifier().fit(test[FEATS], y_test)

model = LGBMClassifier(
objective='binary',
num_boost_round=500,
early_stopping_rounds=100
)

model.fit(
train[FEATS], y_train,
eval_set=[(train[FEATS], y_train), (test[FEATS], y_test)],
eval_metric='binary_logloss',
verbose=100
)

preds = model.predict_proba(test[FEATS])[:, 1]
acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_test, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

/opt/conda/envs/dkt/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/envs/dkt/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[100]	valid_0's binary_logloss: 0.473688	valid_1's binary_logloss: 0.524646
[200]	valid_0's binary_logloss: 0.469613	valid_1's binary_logloss: 0.522083
[300]	valid_0's binary_logloss: 0.466806	valid_1's binary_logloss: 0.520194
[400]	valid_0's binary_logloss: 0.464281	valid_1's binary_logloss: 0.519301
[500]	valid_0's binary_logloss: 0.462104	valid_1's binary_logloss: 0.519008
VALID AUC : 0.8208159411647784 ACC : 0.7463876432486298



In [15]:
# Optuna Libraries
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

# train_test_split
from sklearn.model_selection import train_test_split

from optuna.integration import LightGBMPruningCallback


# random sampler
sampler = TPESampler(seed=10)

# define function
def objective(trial):

    params = {
        'objective': 'binary',
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', -1, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0)
    }

    # Generate model
    model = LGBMClassifier(**params)
    model.fit(
        train[FEATS], y_train,
        eval_set=[(train[FEATS], y_train), (test[FEATS], y_test)],
        eval_metric='auc',
        verbose=100
        )
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    AUC = roc_auc_score(y_test, model.predict_proba(test[FEATS])[:, 1])
    return AUC

optuna_model = optuna.create_study(direction='maximize', study_name='LGBMClassifier', sampler=sampler)
optuna_model.optimize(objective, n_trials=50)

[I 2023-05-11 23:59:30,979] A new study created in memory with name: LGBMClassifier
/opt/conda/envs/dkt/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-05-11 23:59:48,140] Trial 0 finished with value: 0.7952246074339098 and parameters: {'num_leaves': 396, 'min_child_samples': 6, 'max_depth': 63, 'learning_rate': 0.7513158437132257, 'subsample': 0.5486563110723314, 'colsample_bytree': 0.30231698097776294, 'reg_alpha': 0.19806286475962398, 'reg_lambda': 0.7605307121989587}. Best is trial 0 with value: 0.7952246074339098.


[100]	valid_0's auc: 0.880211	valid_0's binary_logloss: 0.404838	valid_1's auc: 0.795225	valid_1's binary_logloss: 0.558574


/opt/conda/envs/dkt/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-05-12 00:00:02,742] Trial 1 finished with value: 0.8020691711389386 and parameters: {'num_leaves': 88, 'min_child_samples': 13, 'max_depth': 68, 'learning_rate': 0.9538594127329871, 'subsample': 0.10355343969512301, 'colsample_bytree': 0.5609730370471989, 'reg_alpha': 0.8126209616521135, 'reg_lambda': 0.6125260668293881}. Best is trial 1 with value: 0.8020691711389386.


[100]	valid_0's auc: 0.848249	valid_0's binary_logloss: 0.447761	valid_1's auc: 0.802069	valid_1's binary_logloss: 0.544637


/opt/conda/envs/dkt/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-05-12 00:00:19,132] Trial 2 finished with value: 0.8049552772808587 and parameters: {'num_leaves': 370, 'min_child_samples': 33, 'max_depth': 92, 'learning_rate': 0.7174300255637137, 'subsample': 0.5882899312101352, 'colsample_bytree': 0.22795304284137427, 'reg_alpha': 0.3733407600514692, 'reg_lambda': 0.6741336150663453}. Best is trial 2 with value: 0.8049552772808587.


...

[100]	valid_0's auc: 0.854196	valid_0's binary_logloss: 0.441958	valid_1's auc: 0.817718	valid_1's binary_logloss: 0.52152


/opt/conda/envs/dkt/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-05-12 00:13:09,608] Trial 48 finished with value: 0.809802226197575 and parameters: {'num_leaves': 284, 'min_child_samples': 18, 'max_depth': 11, 'learning_rate': 0.013463816796995154, 'subsample': 0.10125272112157614, 'colsample_bytree': 0.7320009204336166, 'reg_alpha': 0.059314119999684174, 'reg_lambda': 0.5506406257276487}. Best is trial 41 with value: 0.8224587557145697.


[100]	valid_0's auc: 0.824713	valid_0's binary_logloss: 0.501982	valid_1's auc: 0.809802	valid_1's binary_logloss: 0.567515


/opt/conda/envs/dkt/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-05-12 00:13:23,372] Trial 49 finished with value: 0.8140419399721726 and parameters: {'num_leaves': 248, 'min_child_samples': 9, 'max_depth': 5, 'learning_rate': 0.2821966326585219, 'subsample': 0.13520435937332292, 'colsample_bytree': 0.8044734114427132, 'reg_alpha': 0.0001365143300456062, 'reg_lambda': 0.6155049249911634}. Best is trial 41 with value: 0.8224587557145697.


[100]	valid_0's auc: 0.829943	valid_0's binary_logloss: 0.470004	valid_1's auc: 0.814042	valid_1's binary_logloss: 0.527023


In [16]:
model_trial = optuna_model.best_trial
model_trial_params = model_trial.params
print('Best Trial: score {},\nparams {}'.format(model_trial.value, model_trial_params))

Best Trial: score 0.8224587557145697,
params {'num_leaves': 273, 'min_child_samples': 12, 'max_depth': 21, 'learning_rate': 0.09008688308426457, 'subsample': 0.16873213360536907, 'colsample_bytree': 0.7356898221081205, 'reg_alpha': 0.055959364666762546, 'reg_lambda': 0.6087755634865101}


In [17]:
# Modeling fit
model = LGBMClassifier(**model_trial_params, verbose=0)
model.fit(train[FEATS], y_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031765 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


LGBMClassifier(colsample_bytree=0.7356898221081205,
               learning_rate=0.09008688308426457, max_depth=21,
               min_child_samples=12, num_leaves=273,
               reg_alpha=0.055959364666762546, reg_lambda=0.6087755634865101,
               subsample=0.16873213360536907, verbose=0)

## 5. Inference

In [18]:
# LOAD TESTDATA
test_csv_file_path = os.path.join(data_dir, 'test_data.csv')
test_df = pd.read_csv(test_csv_file_path, dtype=dtype, parse_dates=['Timestamp'])

# FEATURE ENGINEERING
test_df = feature_engineering(test_df)
test_df = feature_engineering_test(test_df)


# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df['userID'] != test_df['userID'].shift(-1)]

# DROP ANSWERCODE
test_df = test_df.drop(['answerCode'], axis=1)




In [19]:
# print(set(test_df['assessmentItemID'].unique()) - set(df['assessmentItemID'].unique()))
# print(set(test_df['userID'].unique()) - set(df['userID'].unique()))
# print(set(test_df['testId'].unique()) - set(df['testId'].unique()))
# print(set(test_df['KnowledgeTag'].unique()) - set(df['KnowledgeTag'].unique()))

In [20]:
# MAKE PREDICTION
total_preds = model.predict_proba(test_df[FEATS])[:, 1]

In [22]:
# SAVE OUTPUT
output_dir = 'output/'
write_path = os.path.join(output_dir, "submission.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write('{},{}\n'.format(id,p))

writing prediction : output/submission.csv
